In [1]:
import numpy as np
import time

def read_input_from_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    N, D, A, B = map(int, lines[0].split())
    dayoff = np.zeros((N+1, D+1))
    for i in range(1, N+1):
        numbers = list(map(int, lines[i].split()))
        for j in range(0, len(numbers) - 1):
            dayoff[i][numbers[j]] = 1
    return N, D, A, B, dayoff

In [2]:
def initialize_random_solution(N, D, dayoff, A, B):
    X = np.zeros((N+1, D+1))
    for i in range(1, N+1):
        for j in range(1, D+1):
            if dayoff[i][j] == 1:
                X[i][j] = 0
    for i in range(1, N+1):
        for j in range(1, D+1):
            if dayoff[i][j]:
                continue
            if j> 1 and X[i][j - 1] == 4:
                X[i][j] = 0
            else: 
                X[i][j] = np.random.randint(0, 4)
    for d in range(1, D+1):
        for shift in range(1, 5):
            count = np.sum(X[:, d] == shift)
            while count < A: 
                available = [i for i in range(1, N+1) if X[i][d] == 0 and dayoff[i][d] == 0 and X[i][d-1] != 4]
                if not available:
                    break
                i = np.random.choice(available)
                X[i][d] = shift
                count += 1
            while count > B:
                assigned = np.where(X[:, d] == shift)[0]
                if not assigned.size:
                    break
                i = np.random.choice(assigned)
                X[i][d] = 0
                count -= 1
    return X
        


In [3]:
def evaluate_fitness(X, N, D, A, B, dayoff):
    penalty = 0
    # check vi pham ngay nghi
    for i in range(1, N + 1):
        for d in range(1, D +1):
            if dayoff[i][d] and X[i][d] != 0:
                penalty -= 1000

    # check vi pham so luong nguoi lam
    for d in range(1, D +1):
        for shift in range(1, 5):
            count = np.sum(X[:, d] == shift)
            if count < A:
                penalty -= 100 * (A - count)
            elif count > B:
                penalty -= 100 * (count - B)

    # check vi pham ca dem 
    for i in range(1, N + 1):
        for d in range(1, D):
            if X[i][d] == 4 and X[i][d + 1] != 0:
                penalty -= 1000

    max_night_shift = 0
    for i in range(1, N + 1):
        count = np.sum(X[i] == 4)
        if count > max_night_shift:
            max_night_shift = count
    return -max_night_shift + penalty

In [4]:
def tournament_selection(population, fitness_scores, tournament_size=3):
    selected_indices = np.random.choice(len(population), tournament_size, replace=False)
    best_index = selected_indices[np.argmax(fitness_scores[selected_indices])]
    return population[best_index]

In [5]:
def crossover(parent1, parent2, crossover_rate=0.6):
    child1 = np.copy(parent1)
    child2 = np.copy(parent2)
    for i in range(1, len(parent1)):
        if np.random.rand() > crossover_rate:
            child1[i] = parent2[i]
            child2[i] = parent1[i]
    return child1, child2



In [6]:
def mutate(X, N, D, dayoff, A, B, mutation_rate=0.1):
    X = np.copy(X)  # Tạo bản sao
    # Bước đột biến
    for i in range(1, N + 1):
        for d in range(1, D + 1):
            if np.random.rand() < mutation_rate:
                if dayoff[i][d] == 1:
                    X[i][d] = 0
                    continue
                if d > 1 and X[i][d - 1] == 4:
                    X[i][d] = 0
                else:
                    X[i][d] = np.random.randint(0, 5)  # Sửa: Bao gồm ca 4
                    if X[i][d] == 4 and d < D:
                        X[i][d + 1] = 0
    # Bước sửa lỗi ca đêm
    for i in range(1, N + 1):
        for d in range(1, D):
            if X[i][d] == 4 and X[i][d + 1] != 0:
                X[i][d + 1] = 0
    # Bước điều chỉnh A <= số nhân viên mỗi ca <= B
    for d in range(1, D + 1):
        for shift in range(1, 5):
            count = np.sum(X[:, d] == shift)
            while count < A:
                available = [i for i in range(1, N + 1) if X[i][d] == 0 and dayoff[i][d] == 0 and (d == 1 or X[i][d - 1] != 4)]
                if not available:
                    break
                i = np.random.choice(available)
                X[i][d] = shift
                if shift == 4 and d < D:  # Thêm: Nếu gán ca đêm, ngày tiếp theo phải nghỉ
                    X[i][d + 1] = 0
                count += 1
            while count > B:
                assigned = np.where(X[:, d] == shift)[0]
                if not assigned.size:
                    break
                i = np.random.choice(assigned)
                X[i][d] = 0
                count -= 1
    # Bước sửa lỗi ca đêm lần cuối
    for i in range(1, N + 1):
        for d in range(1, D):
            if X[i][d] == 4 and X[i][d + 1] != 0:
                X[i][d + 1] = 0
    return X

In [7]:
def genetic_algorithm(N, D, A, B, dayoff):
    population_size = 50
    generations = 20
    mutation_rate = 0.1
    population = [initialize_random_solution(N, D, dayoff, A, B) for _ in range(population_size)]

    for generation in range(generations):
        fitness_scores = np.array([evaluate_fitness(individual, N, D, A, B, dayoff) for individual in population])
        best_index = np.argmax(fitness_scores)
        best_solution = population[best_index]
        new_population = [best_solution]
        
        while len(new_population) < population_size:
            parent1 = tournament_selection(population, fitness_scores)
            parent2 = tournament_selection(population, fitness_scores)
            child1, child2 = crossover(parent1, parent2)
            new_population.append(mutate(child1, N, D, dayoff, A, B, mutation_rate))
            new_population.append(mutate(child2, N, D, dayoff, A, B, mutation_rate))

        population = new_population[:population_size]

        if generation % 10 == 0:
            best_fitness = max(fitness_scores)
            print(f"Generation {generation}: Best Fitness = {best_fitness}")
    best_index = np.argmax(fitness_scores)
    best_solution = population[best_index]
    return best_solution


In [8]:
if __name__ == "__main__":
    filename = 'Testcase/tc'
    output = 'Output/output'
    list_test = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    for tc in list_test:
        start_time = time.time()
        input_file = f"{filename}{tc}.txt"
        output_file = f"{output}{tc}.txt"

        N, D, A, B, dayoff = read_input_from_file(input_file)
        best_solution = genetic_algorithm(N, D, A, B, dayoff)
        fitness = evaluate_fitness(best_solution, N, D, A, B, dayoff)
        with open(output_file, 'w') as file:
            for i in range(1, N + 1):
                file.write(' '.join([str(int(best_solution[i][d])) for d in range(1, D + 1)]) + '\n')
            end_time = time.time()
            file.write("Execution time: {:.2f} seconds\n".format(end_time - start_time))
        print(f"Test case {tc} completed in {end_time - start_time:.2f} seconds.")

Generation 0: Best Fitness = -3
Generation 10: Best Fitness = -1
Test case 1 completed in 0.46 seconds.
Generation 0: Best Fitness = -577209
Generation 10: Best Fitness = -8
Test case 2 completed in 20.80 seconds.
Generation 0: Best Fitness = -1699515
Generation 10: Best Fitness = -13
Test case 3 completed in 71.14 seconds.
Generation 0: Best Fitness = -25104
Generation 10: Best Fitness = -3
Test case 4 completed in 1.88 seconds.
Generation 0: Best Fitness = -2751014
Generation 10: Best Fitness = -13
Test case 5 completed in 123.83 seconds.
Generation 0: Best Fitness = -6383527
Generation 10: Best Fitness = -25
Test case 6 completed in 390.20 seconds.
Generation 0: Best Fitness = -4203
Generation 10: Best Fitness = -2
Test case 7 completed in 0.95 seconds.
Generation 0: Best Fitness = -1680013
Generation 10: Best Fitness = -11
Test case 8 completed in 71.25 seconds.
Generation 0: Best Fitness = -281408
Generation 10: Best Fitness = -8
Test case 9 completed in 10.40 seconds.
Generation 